# Clustering financial data using dataframe

### Importing MLlib libraries

In [1]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._

### Read data and pre-processing

The data that we are going to use in this example is stock market data with the ConnorsRSI indicator. ConnorsRSI is a composite indicator made up from RSI_CLOSE_3, PERCENT_RANK_100, and RSI_STREAK_2. We will use these attributes as well as the actual ConnorsRSI (CRSI) and RSI2 to pass into  KMeans algorithm. The calculation of this data is already normalized from 0 to 100.
The other columns like ID, LABEL, RTN5, FIVE_DAY_GL, and CLOSE we will use to do further analysis once we cluster the instances. They will not be passed into the KMeans algorithm.

In [2]:
// load file and remove header

val data = sc.textFile("data/spykmeans.csv")
val header = data.first
val rows = data.filter(l => l != header)

In [3]:
// define case class

case class CC1(ID: String, LABEL: String, RTN5: Double, FIVE_DAY_GL: Double,
CLOSE: Double, RSI2: Double, RSI_CLOSE_3: Double, PERCENT_RANK_100: Double, RSI_STREAK_2: Double, CRSI: Double)

// comma separator split

val allSplit = rows.map(line => line.split(","))

// map parts to case class

val allData = allSplit.map( p => CC1( p(0).toString, p(1).toString, p(2).trim.toDouble,
p(3).trim.toDouble, p(4).trim.toDouble, p(5).trim.toDouble, p(6).trim.toDouble,
p(7).trim.toDouble, p(8).trim.toDouble, p(9).trim.toDouble))
 

### RDD / Dataframe conversions

In [4]:
// convert rdd to dataframe
import sqlContext.implicits._
val allDF = allData.toDF()

allDF.show()
allDF.schema
// convert back to rdd and cache the data

val rowsRDD = allDF.rdd.map(r => (r.getString(0), r.getString(1), r.getDouble(2),
                                  r.getDouble(3), r.getDouble(4), r.getDouble(5), r.getDouble(6),
                                  r.getDouble(7), r.getDouble(8), r.getDouble(9) ))

rowsRDD.cache()

// convert data to RDD which will be passed to KMeans and cache the data. We are passing in RSI2, RSI_CLOSE_3, PERCENT_RANK_100,
// RSI_STREAK_2 and CRSI to KMeans. These are the attributes we want to use to assign the instance to a cluster

val vectors = allDF.rdd.map(r => Vectors.dense( r.getDouble(5), r.getDouble(6),
                                               r.getDouble(7), r.getDouble(8), r.getDouble(9) ))

vectors.cache()

+--------------+-----+---------+-----------+------+--------+-----------+----------------+------------+-------+
|            ID|LABEL|     RTN5|FIVE_DAY_GL| CLOSE|    RSI2|RSI_CLOSE_3|PERCENT_RANK_100|RSI_STREAK_2|   CRSI|
+--------------+-----+---------+-----------+------+--------+-----------+----------------+------------+-------+
|2015-09-16:SPY|   UP|  2.76708|   -3.28704|200.18| 91.5775|     81.572|            84.0|     73.2035|79.5918|
|2015-09-15:SPY|   UP| 0.521704|   -2.29265|198.46| 83.4467|    72.9477|            92.0|     60.6273|75.1917|
|2015-09-14:SPY|   DN|  1.77579|    0.22958|196.01| 47.0239|    51.3076|            31.0|      25.807|36.0382|
|2015-09-11:SPY|   UP|  0.60854|  -0.655688|196.74| 69.9559|    61.0005|            76.0|      76.643|71.2145|
|2015-09-10:SPY|   UP| 0.225168|    1.98111|195.85| 57.2462|    53.9258|            79.0|     65.2266|66.0508|
|2015-09-09:SPY|   DN|   1.5748|    2.76708|194.79| 42.8488|    46.1728|             7.0|     31.9797|28.3842|
|

MapPartitionsRDD[151] at map at <console>:56

In [5]:
allDF.schema

StructType(StructField(ID,StringType,true), StructField(LABEL,StringType,true), StructField(RTN5,DoubleType,false), StructField(FIVE_DAY_GL,DoubleType,false), StructField(CLOSE,DoubleType,false), StructField(RSI2,DoubleType,false), StructField(RSI_CLOSE_3,DoubleType,false), StructField(PERCENT_RANK_100,DoubleType,false), StructField(RSI_STREAK_2,DoubleType,false), StructField(CRSI,DoubleType,false))

### Run cluster analysis

In [6]:
//KMeans model with 2 clusters and 20 iterations

val kMeansModel = KMeans.train(vectors, 2, 20)

//Print the center of each cluster

kMeansModel.clusterCenters.foreach(println)

// Get the prediction from the model with the ID so we can link them back to other information

val predictions = rowsRDD.map{r => (r._1, kMeansModel.predict(Vectors.dense(r._6, r._7, r._8, r._9, r._10) ))}

// convert the rdd to a dataframe

val predDF = predictions.toDF("ID", "CLUSTER")

[79.06854271582732,72.01910089928057,70.35431654676259,72.65608415467624,71.67650269784168]
[25.590238191780806,33.762228675799086,22.24429223744292,25.428050107305936,27.14485747716894]


### Join the dataframes on ID

In [7]:
val t = allDF.join(predDF, "ID")
t.printSchema()

root
 |-- ID: string (nullable = true)
 |-- LABEL: string (nullable = true)
 |-- RTN5: double (nullable = false)
 |-- FIVE_DAY_GL: double (nullable = false)
 |-- CLOSE: double (nullable = false)
 |-- RSI2: double (nullable = false)
 |-- RSI_CLOSE_3: double (nullable = false)
 |-- PERCENT_RANK_100: double (nullable = false)
 |-- RSI_STREAK_2: double (nullable = false)
 |-- CRSI: double (nullable = false)
 |-- CLUSTER: integer (nullable = false)



### Review a subset of each cluster

In [8]:
t.filter("CLUSTER = 0").show()
t.filter("CLUSTER = 1").show()

+--------------+-----+----------+-----------+------+-------+-----------+----------------+------------+-------+-------+
|            ID|LABEL|      RTN5|FIVE_DAY_GL| CLOSE|   RSI2|RSI_CLOSE_3|PERCENT_RANK_100|RSI_STREAK_2|   CRSI|CLUSTER|
+--------------+-----+----------+-----------+------+-------+-----------+----------------+------------+-------+-------+
|2012-05-24:SPY|   UP|   1.27617|   -3.29737|132.53|80.5069|    61.8344|            60.0|     95.4215|72.4186|      0|
|2012-08-16:SPY|   UP|  0.981438|  -0.929643|141.99|99.1446|    96.5522|            79.0|     70.8234|82.1252|      0|
|2014-01-15:SPY|   UP|  0.621186|   -1.01267|184.66|76.9431|      69.09|            78.0|     64.3179|70.4693|      0|
|2008-04-25:SPY|   UP|  0.808781|    1.36819| 139.6|92.4863|    86.0256|            81.0|     72.3651|79.7969|      0|
|2008-07-17:SPY|   UP|-0.0798085|   0.247604| 125.2|79.5427|    66.7112|            85.0|     85.0435|78.9182|      0|
|2010-09-01:SPY|   UP|    2.3787|    2.26812|108

### Get descriptive statistics for each cluster

In [9]:
t.filter("CLUSTER = 0").describe("RTN5","FIVE_DAY_GL","CLOSE","RSI2","RSI_CLOSE_3").show()
t.filter("CLUSTER = 0").describe("PERCENT_RANK_100","RSI_STREAK_2","CRSI","CLUSTER").show()
t.filter("CLUSTER = 1").describe("RTN5","FIVE_DAY_GL","CLOSE","RSI2","RSI_CLOSE_3").show()
t.filter("CLUSTER = 1").describe("PERCENT_RANK_100","RSI_STREAK_2","CRSI","CLUSTER").show()

+-------+-----------------+--------------------+------------------+-----------------+------------------+
|summary|             RTN5|         FIVE_DAY_GL|             CLOSE|             RSI2|       RSI_CLOSE_3|
+-------+-----------------+--------------------+------------------+-----------------+------------------+
|  count|             1112|                1112|              1112|             1112|              1112|
|   mean|1.150415319037769|-0.03837807874999998|143.98284172661872|79.06854271582732| 72.01910089928056|
| stddev|2.261222536974694|  2.4702502060751694| 36.14835678821346|16.75622556167194|16.707962903743002|
|    min|         -8.19672|            -17.2425|             71.73|          20.3873|           19.0374|
|    max|          19.4036|             10.0358|             213.5|          99.9975|           99.8326|
+-------+-----------------+--------------------+------------------+-----------------+------------------+

+-------+------------------+------------------+-------

checked